In [98]:
import requests
import csv
import pandas as pd
import io
import time
import glob
import os
import re

In [10]:
# Only for one city and state

url = "https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.csv?api_key=vhzR3aV1Q4BjnUTcI2IOADKPCQTnQPbVKvC5HKMd"

# payload = "names=2012&leap_day=false&interval=60&utc=false&full_name=Honored%2BUser&email=honored.user%40gmail.com&affiliation=NREL&mailing_list=true&reason=Academic&attributes=dhi%2Cdni%2Cwind_speed%2Cair_temperature&wkt=MULTIPOINT(-106.22%2032.9741%2C-106.18%2032.9741%2C-106.1%2032.9741)"

payload ="names=2020&leap_day=false&interval=60&utc=false&full_name=Kasha%2BMuzila&email=kashamuzila@gmail.com&affiliation=NREL&mailing_list=false&reason=Academic&attributes=dhi%2Cghi%2Cclearsky_dhi%2Cclearsky_ghi%2Ccloud_type%2Cdew_point%2Crelative_humidity%2Cwind_direction%2Cwind_speed%2Cair_temperature&wkt=POINT(-109.2225+41.5908)"

headers = {
    'content-type': "application/x-www-form-urlencoded",
    'cache-control': "no-cache"
}

response = requests.request("POST", url, data=payload, headers=headers)

time.sleep(1.5)

with open('Data/NSRDB_data/sh_WY_Roc_2020.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for line in response.iter_lines():
        writer.writerow(line.decode('utf-8').split(','))

print(response.text)

Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation,Local Time Zone,Clearsky DHI Units,Clearsky DNI Units,Clearsky GHI Units,Dew Point Units,DHI Units,DNI Units,GHI Units,Solar Zenith Angle Units,Temperature Units,Pressure Units,Relative Humidity Units,Precipitable Water Units,Wind Direction Units,Wind Speed Units,Cloud Type -15,Cloud Type 0,Cloud Type 1,Cloud Type 2,Cloud Type 3,Cloud Type 4,Cloud Type 5,Cloud Type 6,Cloud Type 7,Cloud Type 8,Cloud Type 9,Cloud Type 10,Cloud Type 11,Cloud Type 12,Fill Flag 0,Fill Flag 1,Fill Flag 2,Fill Flag 3,Fill Flag 4,Fill Flag 5,Surface Albedo Units,Version
NSRDB,168574,-,-,-,41.61,-109.22,-7,1936,-7,w/m2,w/m2,w/m2,c,w/m2,w/m2,w/m2,Degree,c,mbar,%,cm,Degrees,m/s,N/A,Clear,Probably Clear,Fog,Water,Super-Cooled Water,Mixed,Opaque Ice,Cirrus,Overlapping,Overshooting,Unknown,Dust,Smoke,N/A,Missing Image,Low Irradiance,Exceeds Clearsky,Missing CLoud Properties,Rayleigh Violation,N/A,3.2.0
Year,Month,Day,Hour,Minute,DHI,GHI,Cle

In [47]:
states = ('AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY')
dict_states_cities = {'AL': ['Birmingham', 'Montgomery', 'Mobile', 'Huntsville', 'Tuscaloosa'],
         'AK': ['Anchorage', 'Fairbanks', 'Juneau', 'Sitka', 'Ketchikan'],
         'AZ': ['Phoenix', 'Tucson', 'Mesa', 'Chandler', 'Glendale'],
         'AR': ['Little Rock', 'Fort Smith', 'North Little Rock', 'Fayetteville', 'Springdale'],
         'CA': ['Los Angeles', 'San Diego', 'San Jose', 'San Francisco', 'Fresno'],
         'CO': ['Denver', 'Colorado Springs', 'Aurora', 'Fort Collins', 'Lakewood'],
         'CT': ['Bridgeport', 'New Haven', 'Hartford', 'Stamford', 'Waterbury'],
         'DE': ['Wilmington', 'Dover', 'Newark', 'Middletown', 'Smyrna'],
         'FL': ['Jacksonville', 'Miami', 'Tampa', 'St. Petersburg', 'Orlando'],
         'GA': ['Atlanta', 'Augusta-Richmond', 'Columbus', 'Savannah', 'Athens'],
         'HI': ['Honolulu', 'Hilo', 'Kailua', 'Kapolei', 'Kaneohe'],
         'ID': ['Boise', 'Nampa', 'Meridian', 'Idaho Falls', 'Pocatello'],
         'IL': ['Chicago', 'Aurora', 'Rockford', 'Joliet', 'Naperville'],
         'IN': ['Indianapolis', 'Fort Wayne', 'Evansville', 'South Bend', 'Hammond'],
         'IA': ['Des Moines', 'Cedar Rapids', 'Davenport', 'Sioux City', 'Waterloo'],
         'KS': ['Wichita', 'Overland Park', 'Kansas City', 'Topeka, Olathe'],
         'KY': ['Louisville', 'Lexington', 'Bowling Green', 'Owensboro', 'Covington'],
         'LA': ['New Orleans', 'Baton Rouge', 'Shreveport', 'Lafayette', 'Lake Charles'],
         'ME': ['Portland', 'Lewiston', 'Bangor', 'South Portland', 'Auburn'],
         'MD': ['Baltimore', 'Frederick', 'Rockville', 'Gaithersburg', 'Bowie'],
         'MA': ['Boston', 'Worcester', 'Springfield', 'Lowell', 'Cambridge'],
         'MI': ['Detroit', 'Grand Rapids', 'Warren', 'Sterling Heights', 'Lansing'],
         'MN': ['Minneapolis', 'Saint Paul', 'Rochester', 'Duluth', 'Bloomington'],
         'MS': ['Jackson', 'Gulfport', 'Hattiesburg', 'Southaven', 'Biloxi'],
         'MO': ['Kansas City', 'Saint Louis', 'Springfield', 'Independence', 'Columbia'],
         'MT': ['Billings', 'Missoula', 'Great Falls', 'Bozeman', 'Butte'],
         'NE': ['Omaha', 'Lincoln', 'Bellevue', 'Grand Island', 'Kearney'],
         'NV': ['Las Vegas', 'Henderson', 'North Las Vegas', 'Reno', 'Sparks'],
         'NH': ['Manchester', 'Nashua', 'Concord', 'Derry', 'Rochester'],
         'NJ': ['Newark', 'Jersey City', 'Paterson', 'Elizabeth', 'Edison'],
         'NM': ['Albuquerque', 'Las Cruces', 'Rio Rancho', 'Santa Fe', 'Roswell'],
         'NY': ['New York', 'Buffalo', 'Rochester', 'Yonkers', 'Syracuse'],
         'NC': ['Charlotte', 'Raleigh', 'Greensboro', 'Winston-Salem', 'Durham'],
         'ND': ['Fargo', 'Bismarck', 'Grand Forks', 'Minot', 'West Fargo'],
         'OH': ['Columbus', 'Cleveland', 'Cincinnati', 'Toledo', 'Akron'],
         'OK': ['Oklahoma City', 'Tulsa', 'Norman', 'Broken Arrow', 'Lawton'],
         'OR': ['Portland', 'Eugene', 'Salem', 'Gresham', 'Hillsboro'],
         'PA': ['Philadelphia', 'Pittsburgh', 'Allentown', 'Erie', 'Reading'],
         'RI': ['Providence', 'Warwick', 'Cranston', 'Pawtucket', 'East Providence'],
         'SC': ['Columbia', 'Charleston', 'North Charleston', 'Mount Pleasant', 'Rock Hill'],
         'SD': ['Sioux Falls', 'Rapid City', 'Aberdeen', 'Brookings', 'Watertown'],
         'TN': ['Memphis', 'Nashville', 'Knoxville', 'Chattanooga', 'Clarksville'],
         'TX': ['Houston', 'San Antonio', 'Dallas', 'Austin', 'Fort Worth'],
         'UT': ['Salt Lake City', 'West Valley City', 'Provo', 'West Jordan', 'Orem'],
         'VT': ['Burlington', 'Essex', 'South Burlington', 'Colchester', 'Rutland'],
         'VA': ['Virginia Beach', 'Norfolk', 'Chesapeake', 'Richmond', 'Newport News'],
         'WA': ['Seattle', 'Spokane', 'Tacoma', 'Vancouver', 'Bellevue'],
         'WV': ['Charleston', 'Huntington', 'Parkersburg', 'Morgantown', 'Wheeling'],
         'WI': ['Milwaukee', 'Madison', 'Green Bay', 'Kenosha', 'Racine'],
         'WY': ['Cheyenne', 'Casper', 'Laramie', 'Gillette', 'Rock Springs']}

In [48]:
# df_states_cities = pd.DataFrame.from_dict(dict_states_cities)
df_states_cities = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dict_states_cities.items() ]))
df_states_cities.head()

,AL,AK,AZ,AR,CA,CO,CT,DE,FL,GA,HI,ID,IL,IN,IA,KS,KY,LA,ME,MD,MA,MI,MN,MS,MO,MT,NE,NV,NH,NJ,NM,NY,NC,ND,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY
0,Birmingham,Anchorage,Phoenix,Little Rock,Los Angeles,Denver,Bridgeport,Wilmington,Jacksonville,Atlanta,Honolulu,Boise,Chicago,Indianapolis,Des Moines,Wichita,Louisville,New Orleans,Portland,Baltimore,Boston,Detroit,Minneapolis,Jackson,Kansas City,Billings,Omaha,Las Vegas,Manchester,Newark,Albuquerque,New York,Charlotte,Fargo,Columbus,Oklahoma City,Portland,Philadelphia,Providence,Columbia,Sioux Falls,Memphis,Houston,Salt Lake City,Burlington,Virginia Beach,Seattle,Charleston,Milwaukee,Cheyenne
1,Montgomery,Fairbanks,Tucson,Fort Smith,San Diego,Colorado Springs,New Haven,Dover,Miami,Augusta-Richmond,Hilo,Nampa,Aurora,Fort Wayne,Cedar Rapids,Overland Park,Lexington,Baton Rouge,Lewiston,Frederick,Worcester,Grand Rapids,Saint Paul,Gulfport,Saint Louis,Missoula,Lincoln,Henderson,Nashua,Jersey City,Las Cruces,Buffalo,Raleigh,Bismarck,Cleveland,Tulsa,Eugene,Pittsburgh,Warwick,Charleston,Rapid City,Nashville,San Antonio,West Valley City,Essex,Norfolk,Spokane,Huntington,Madison,Casper
2,Mobile,Juneau,Mesa,North Little Rock,San Jose,Aurora,Hartford,Newark,Tampa,Columbus,Kailua,Meridian,Rockford,Evansville,Davenport,Kansas City,Bowling Green,Shreveport,Bangor,Rockville,Springfield,Warren,Rochester,Hattiesburg,Springfield,Great Falls,Bellevue,North Las Vegas,Concord,Paterson,Rio Rancho,Rochester,Greensboro,Grand Forks,Cincinnati,Norman,Salem,Allentown,Cranston,North Charleston,Aberdeen,Knoxville,Dallas,Provo,South Burlington,Chesapeake,Tacoma,Parkersburg,Green Bay,Laramie
3,Huntsville,Sitka,Chandler,Fayetteville,San Francisco,Fort Collins,Stamford,Middletown,St. Petersburg,Savannah,Kapolei,Idaho Falls,Joliet,South Bend,Sioux City,"Topeka, Olathe",Owensboro,Lafayette,South Portland,Gaithersburg,Lowell,Sterling Heights,Duluth,Southaven,Independence,Bozeman,Grand Island,Reno,Derry,Elizabeth,Santa Fe,Yonkers,Winston-Salem,Minot,Toledo,Broken Arrow,Gresham,Erie,Pawtucket,Mount Pleasant,Brookings,Chattanooga,Austin,West Jordan,Colchester,Richmond,Vancouver,Morgantown,Kenosha,Gillette
4,Tuscaloosa,Ketchikan,Glendale,Springdale,Fresno,Lakewood,Waterbury,Smyrna,Orlando,Athens,Kaneohe,Pocatello,Naperville,Hammond,Waterloo,NaN,Covington,Lake Charles,Auburn,Bowie,Cambridge,Lansing,Bloomington,Biloxi,Columbia,Butte,Kearney,Sparks,Rochester,Edison,Roswell,Syracuse,Durham,West Fargo,Akron,Lawton,Hillsboro,Reading,East Providence,Rock Hill,Watertown,Clarksville,Fort Worth,Orem,Rutland,Newport News,Bellevue,Wheeling,Racine,Rock Springs


In [4]:
df_cities_states_lat_long = pd.read_csv('df.csv')
df_cities_states_lat_long.head()

,state,city,lat,long,state_name,country
0,AL,Birmingham,33.5250,-86.8130,Alabama,United States
1,AL,Montgomery,32.3617,-86.2792,Alabama,United States
2,AL,Mobile,30.6889,-88.0448,Alabama,United States
3,AL,Huntsville,34.7295,-86.5853,Alabama,United States
4,AL,Tuscaloosa,33.2065,-87.5346,Alabama,United States


In [5]:
df_cities_states_lat_long['city_abbr'] = df_cities_states_lat_long['city'].str[:3]
# df.insert(len(df.columns), 'country', "United States")
df_cities_states_lat_long.head()

,state,city,lat,long,state_name,country,city_abbr
0,AL,Birmingham,33.5250,-86.8130,Alabama,United States,Bir
1,AL,Montgomery,32.3617,-86.2792,Alabama,United States,Mon
2,AL,Mobile,30.6889,-88.0448,Alabama,United States,Mob
3,AL,Huntsville,34.7295,-86.5853,Alabama,United States,Hun
4,AL,Tuscaloosa,33.2065,-87.5346,Alabama,United States,Tus


In [62]:
type(df_cities_states_lat_long['city'][1])

str

In [11]:
# Pulls all city and states 

for i in df_cities_states_lat_long.index: 
    url = "https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.csv?api_key=vhzR3aV1Q4BjnUTcI2IOADKPCQTnQPbVKvC5HKMd"

    # payload = "names=2012&leap_day=false&interval=60&utc=false&full_name=Honored%2BUser&email=honored.user%40gmail.com&affiliation=NREL&mailing_list=true&reason=Academic&attributes=dhi%2Cdni%2Cwind_speed%2Cair_temperature&wkt=MULTIPOINT(-106.22%2032.9741%2C-106.18%2032.9741%2C-106.1%2032.9741)"

    payload ="names=2017&leap_day=false&interval=60&utc=false&full_name=Kasha%2BMuzila&email=kashamuzila@gmail.com&affiliation=NREL&mailing_list=false&reason=Academic&attributes=dhi%2Cghi%2Cclearsky_dhi%2Cclearsky_ghi%2Ccloud_type%2Cdew_point%2Crelative_humidity%2Cwind_direction%2Cwind_speed%2Cair_temperature&wkt=POINT({}+{})".format(df_cities_states_lat_long['long'][i], df_cities_states_lat_long['lat'][i])

    headers = {
        'content-type': "application/x-www-form-urlencoded",
        'cache-control': "no-cache"
    }

    response = requests.request("POST", url, data=payload, headers=headers)
    
    # print(response.text)
    time.sleep(1.5)
    
    csv_name = 'Data/2017_data/sh_{}_{}_2017.csv'.format(df_cities_states_lat_long['state'][i], df_cities_states_lat_long['city'][i][:3])
    
    with open(csv_name, 'w', newline='') as f:
        writer = csv.writer(f)
        for line in response.iter_lines():
            writer.writerow(line.decode('utf-8').split(','))
    
    # break
        
    time.sleep(0.5)

    # print(response.text)
    # print(df_cities_states_lat_long['lat'][i], df_cities_states_lat_long['long'][i])

In [57]:
# DO NOT USE -- ALREADY RAN THIS
# Deletes the first 2 rows in all csvs

# base_directory = 'Data/fix'    
# for dir_path, dir_name_list, file_name_list in os.walk(base_directory):
#     for file_name in file_name_list:
#         # If this is not a CSV file
#         if not file_name.endswith('.csv'):
#             # Skip it
#             continue
#         file_path = os.path.join(dir_path, file_name)
#         with open(file_path, 'r') as ifile:
#             line_list = ifile.readlines()
#         with open(file_path, 'w') as ofile:
#             ofile.writelines(line_list[2:])

In [13]:
# df_cities_states_lat_long.head()
df_cities_states_lat_long[df_cities_states_lat_long['state'] == 'AK']

,state,city,lat,long,state_name,country,city_abbr
5,AK,Anchorage,61.2166,-149.8996,Alaska,United States,Anc
6,AK,Fairbanks,64.8436,-147.7231,Alaska,United States,Fai
7,AK,Juneau,58.3019,-134.4197,Alaska,United States,Jun
8,AK,Sitka,57.0531,-135.3300,Alaska,United States,Sit
9,AK,Ketchikan,55.3500,-131.6733,Alaska,United States,Ket


In [87]:
# Appends coloumns from df_cities_states_lat_long to all csvs

base_directory = 'Data/fix'  
# for i in df_cities_states_lat_long.index: 
for dir_path, dir_name_list, file_name_list in os.walk(base_directory):
    for file_name in file_name_list:
        # If this is not a CSV file
        if not file_name.endswith('.csv'):
            # Skip it
            continue
        print('Editing file ' + file_name + '...')
        file_path = os.path.join(dir_path, file_name)
        print(file_path)
        try:
            st_abbr = file_name.split('_')[1]
            ct_abbr = file_name.split('_')[2]
            row = df_cities_states_lat_long[(df_cities_states_lat_long['state'] == st_abbr) & (df_cities_states_lat_long['city_abbr'] == ct_abbr)]
            df = pd.read_csv(file_path)
            df.insert(len(df.columns), 'state', row['state_name'].values[0])
            df.insert(len(df.columns), 'city', row['city'].values[0])
            df.insert(len(df.columns), 'country', row['country'].values[0])
            df.insert(len(df.columns), 'lat', (row['lat'].values[0]))
            df.insert(len(df.columns), 'long', row['long'].values[0])
            df.to_csv(file_path)
        except pd.errors.EmptyDataError:
            print(file_name + ' is empty')
        except ValueError:
            print(file_name + ' already has column')
print('dunzo')

dunzo


In [37]:
type(dict_states_cities)

dict

In [59]:
base_directory = 'Data/NSRDB_data'  
# for i in df_cities_states_lat_long.index: 
for dir_path, dir_name_list, file_name_list in os.walk(base_directory):
    for file_name in file_name_list:
        # If this is not a CSV file
        if not file_name.endswith('.csv'):
            # Skip it
            continue
        print('Editing file ' + file_name + '...')
        file_path = os.path.join(dir_path, file_name)
        # print(file_path)
        try:
            st_abbr = file_name.split('_')[1]
            ct_abbr = file_name.split('_')[2]
            csv = file_name.split('_')[3]
            year = csv[:-4]
            row = df_cities_states_lat_long[(df_cities_states_lat_long['state'] == st_abbr) & (df_cities_states_lat_long['city_abbr'] == ct_abbr)]
            # print(row)
            df = pd.read_csv(file_path)
            agg_group_by = df.groupby('Month') \
                            .agg(month_DHI = ('DHI', 'sum'), \
                                 month_GHI = ('GHI', 'sum'), \
                                 month_clearsky_DHI = ('Clearsky DHI', 'sum'), \
                                 month_clearsky_GHI = ('Clearsky GHI', 'sum'), \
                                 min_cloud_type = ('Cloud Type', 'min'), \
                                 max_cloud_type = ('Cloud Type', 'max'), \
                                 ave_cloud_type = ('Cloud Type', 'mean'), \
                                 min_dew_point = ('Dew Point', 'min'), \
                                 max_dew_point = ('Dew Point', 'max'), \
                                 ave_dew_point = ('Dew Point', 'mean'), \
                                 min_rel_humid = ('Relative Humidity', 'min'), \
                                 max_rel_humid = ('Relative Humidity', 'max'), \
                                 ave_rel_humid = ('Relative Humidity', 'mean'), \
                                 min_wind_dir = ('Wind Direction', 'min'), \
                                 max_wind_dir = ('Wind Direction', 'max'), \
                                 ave_wind_dir = ('Wind Direction', 'mean'), \
                                 min_wind_speed = ('Wind Speed', 'min'), \
                                 max_wind_speed = ('Wind Speed', 'max'), \
                                 ave_wind_speed = ('Wind Speed', 'mean'), \
                                 min_temp = ('Temperature', 'min'), \
                                 max_temp = ('Temperature', 'max'), \
                                 ave_temp = ('Temperature', 'mean')).reset_index()
            agg_group_by.insert(len(agg_group_by.columns), 'year', year)
            agg_group_by.insert(len(agg_group_by.columns), 'state', row['state_name'].values[0])
            agg_group_by.insert(len(agg_group_by.columns), 'city', row['city'].values[0])
            agg_group_by.insert(len(agg_group_by.columns), 'country', row['country'].values[0])
            agg_group_by.insert(len(agg_group_by.columns), 'lat', (row['lat'].values[0]))
            agg_group_by.insert(len(agg_group_by.columns), 'long', row['long'].values[0])
            # print(type(agg_group_by))
            # agg_group_by.reset_index()
            # agg_group_by.to_csv(file_path)
            # print(agg_group_by)
            agg_group_by.to_csv('Data/month_NSRDB_data/month_{}'.format(file_name))
            # print(st_abbr, ct_abbr, csv, year)
            # print(row)
        except pd.errors.EmptyDataError:
            print(file_name + ' is empty')
        except ValueError:
            print(file_name + ' already has column')
print('dunzo')

Editing file sh_AK_Anc_2017.csv...
sh_AK_Anc_2017.csv is empty
Editing file sh_AK_Anc_2018.csv...
sh_AK_Anc_2018.csv is empty
Editing file sh_AK_Anc_2019.csv...
sh_AK_Anc_2019.csv is empty
Editing file sh_AK_Anc_2020.csv...
sh_AK_Anc_2020.csv is empty
Editing file sh_AK_Fai_2017.csv...
sh_AK_Fai_2017.csv is empty
Editing file sh_AK_Fai_2018.csv...
sh_AK_Fai_2018.csv is empty
Editing file sh_AK_Fai_2019.csv...
sh_AK_Fai_2019.csv is empty
Editing file sh_AK_Fai_2020.csv...
sh_AK_Fai_2020.csv is empty
Editing file sh_AK_Jun_2017.csv...
Editing file sh_AK_Jun_2018.csv...
Editing file sh_AK_Jun_2019.csv...
Editing file sh_AK_Jun_2020.csv...
Editing file sh_AK_Ket_2017.csv...
Editing file sh_AK_Ket_2018.csv...
Editing file sh_AK_Ket_2019.csv...
Editing file sh_AK_Ket_2020.csv...
Editing file sh_AK_Sit_2017.csv...
Editing file sh_AK_Sit_2018.csv...
Editing file sh_AK_Sit_2019.csv...
Editing file sh_AK_Sit_2020.csv...
Editing file sh_AL_Bir_2017.csv...
Editing file sh_AL_Bir_2018.csv...
Editin

In [71]:
base_directory = 'Data/month_NSRDB_data'  
# for i in df_cities_states_lat_long.index: 
for dir_path, dir_name_list, file_name_list in os.walk(base_directory):
    for file_name in file_name_list:
        # If this is not a CSV file
        if not file_name.endswith('.csv'):
            # Skip it
            continue
        print('Editing file ' + file_name + '...')
        file_path = os.path.join(dir_path, file_name)
        # print(file_path)
        try:
            df = pd.read_csv(file_path)
            df['ave_cloud_type'] = df['ave_cloud_type'].round()
            df['min_dew_point_F'] = ((df['min_dew_point'] * 1.8) + 32).round()
            df['max_dew_point_F'] = ((df['max_dew_point'] * 1.8) + 32).round()
            df['ave_dew_point_F'] = ((df['ave_dew_point'] * 1.8) + 32)
            df['min_temp_F'] = ((df['min_temp'] * 1.8) + 32).round()
            df['max_temp_F'] = ((df['max_temp'] * 1.8) + 32).round()
            df['ave_temp_F'] = ((df['ave_temp'] * 1.8) + 32)
            df['min_wind_speed_ft-s'] = (df['min_wind_speed'] * 3.28084).round(1)
            df['max_wind_speed_ft-s'] = (df['max_wind_speed'] * 3.28084).round(1)
            df['ave_wind_speed_ft-s'] = (df['ave_wind_speed'] * 3.28084)
            df.to_csv(file_path)
        except pd.errors.EmptyDataError:
            print(file_name + ' is empty')
        except ValueError:
            print(file_name + ' already has column')
print('dunzo')

Editing file month_sh_AK_Jun_2017.csv...
Editing file month_sh_AK_Jun_2018.csv...
Editing file month_sh_AK_Jun_2019.csv...
Editing file month_sh_AK_Jun_2020.csv...
Editing file month_sh_AK_Ket_2017.csv...
Editing file month_sh_AK_Ket_2018.csv...
Editing file month_sh_AK_Ket_2019.csv...
Editing file month_sh_AK_Ket_2020.csv...
Editing file month_sh_AK_Sit_2017.csv...
Editing file month_sh_AK_Sit_2018.csv...
Editing file month_sh_AK_Sit_2019.csv...
Editing file month_sh_AK_Sit_2020.csv...
Editing file month_sh_AL_Bir_2017.csv...
Editing file month_sh_AL_Bir_2018.csv...
Editing file month_sh_AL_Bir_2019.csv...
Editing file month_sh_AL_Bir_2020.csv...
Editing file month_sh_AL_Hun_2017.csv...
Editing file month_sh_AL_Hun_2018.csv...
Editing file month_sh_AL_Hun_2019.csv...
Editing file month_sh_AL_Hun_2020.csv...
Editing file month_sh_AL_Mob_2017.csv...
Editing file month_sh_AL_Mob_2018.csv...
Editing file month_sh_AL_Mob_2019-checkpoint.csv...
Editing file month_sh_AL_Mob_2019.csv...
Editi

In [78]:
base_directory = 'Data/month_NSRDB_data'  
# for i in df_cities_states_lat_long.index: 
for dir_path, dir_name_list, file_name_list in os.walk(base_directory):
    for file_name in file_name_list:
        # If this is not a CSV file
        if not file_name.endswith('.csv'):
            # Skip it
            continue
        print('Editing file ' + file_name + '...')
        file_path = os.path.join(dir_path, file_name)
        # print(file_path)
        try:
            df = pd.read_csv(file_path)
            df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
            # print(df)
            df.to_csv(file_path)
        except pd.errors.EmptyDataError:
            print(file_name + ' is empty')
        except ValueError:
            print(file_name + ' already has column')
print('dunzo')

Editing file month_sh_AK_Jun_2017.csv...
Editing file month_sh_AK_Jun_2018.csv...
Editing file month_sh_AK_Jun_2019.csv...
Editing file month_sh_AK_Jun_2020.csv...
Editing file month_sh_AK_Ket_2017.csv...
Editing file month_sh_AK_Ket_2018.csv...
Editing file month_sh_AK_Ket_2019.csv...
Editing file month_sh_AK_Ket_2020.csv...
Editing file month_sh_AK_Sit_2017.csv...
Editing file month_sh_AK_Sit_2018.csv...
Editing file month_sh_AK_Sit_2019.csv...
Editing file month_sh_AK_Sit_2020.csv...
Editing file month_sh_AL_Bir_2017.csv...
Editing file month_sh_AL_Bir_2018.csv...
Editing file month_sh_AL_Bir_2019.csv...
Editing file month_sh_AL_Bir_2020.csv...
Editing file month_sh_AL_Hun_2017.csv...
Editing file month_sh_AL_Hun_2018.csv...
Editing file month_sh_AL_Hun_2019.csv...
Editing file month_sh_AL_Hun_2020.csv...
Editing file month_sh_AL_Mob_2017.csv...
Editing file month_sh_AL_Mob_2018.csv...
Editing file month_sh_AL_Mob_2019.csv...
Editing file month_sh_AL_Mob_2020.csv...
Editing file mon

In [91]:
# DO NOT RUN ALREADY RAN THIS

# path = r'Data\month_NSRDB_data'
# all_files = glob.glob(path + "/*.csv")
# df_files = (pd.read_csv(f) for f in all_files)
# df = pd.concat(df_files)
# df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
# # print(df)
# df.to_csv('Data/combined_NSRDB_data/combined_NSRDB_data.csv')
# # print(df)

In [99]:
df = pd.read_csv('Data/combined_NSRDB_data/combined_NSRDB_data.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['DHI_per_day'] = (df['month_DHI'] / 365) / 28
df['GHI_per_day'] = (df['month_GHI'] / 365) / 28
df['clearsky_DHI_per_day'] = (df['month_clearsky_DHI'] / 365) / 28
df['clearsky_GHI_per_day'] = (df['month_clearsky_GHI'] / 365) / 28
df.head()
df.to_csv('Data/combined_NSRDB_data/combined_NSRDB_data.csv')

In [26]:
# pd.set_option('display.max_columns', None)
# los_angeles = pd.read_csv('los_angeles.csv')
# los_angeles.head()

,Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation,Local Time Zone,Clearsky DHI Units,Clearsky DNI Units,Clearsky GHI Units,Dew Point Units,DHI Units,DNI Units,GHI Units,Solar Zenith Angle Units,Temperature Units,Pressure Units,Relative Humidity Units,Precipitable Water Units,Wind Direction Units,Wind Speed Units,Cloud Type -15,Cloud Type 0,Cloud Type 1,Cloud Type 2,Cloud Type 3,Cloud Type 4,Cloud Type 5,Cloud Type 6,Cloud Type 7,Cloud Type 8,Cloud Type 9,Cloud Type 10,Cloud Type 11,Cloud Type 12,Fill Flag 0,Fill Flag 1,Fill Flag 2,Fill Flag 3,Fill Flag 4,Fill Flag 5,Surface Albedo Units,Version
0,NSRDB,83165,-,-,-,34.01,-118.26,-8,61,-8,w/m2,w/m2,w/m2,c,w/m2,w/m2,w/m2,Degree,c,mbar,%,cm,Degrees,m/s,NaN,Clear,Probably Clear,Fog,Water,Super-Cooled Water,Mixed,Opaque Ice,Cirrus,Overlapping,Overshooting,Unknown,Dust,Smoke,NaN,Missing Image,Low Irradiance,Exceeds Clearsky,Missing CLoud Properties,Rayleigh Violation,NaN,3.2.0
1,Year,Month,Day,Hour,Minute,DHI,GHI,Clearsky DHI,Clearsky GHI,Cloud Type,Dew Point,Relative Humidity,Wind Direction,Wind Speed,Temperature,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020,1,1,0,30,0,0,0,0,0,1.9000000000000001,61.730000000000004,65,2.6,8.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,1,1,1,30,0,0,0,0,0,1.8,61.480000000000004,60,2.5,8.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,1,1,2,30,0,0,0,0,0,1.7000000000000002,61.800000000000004,50,2.4000000000000004,8.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
